In [1]:
import yfinance as yf
import pandas as pd
import threading
import numpy as np

In [2]:
# Fetching DAX tickers from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/DAX')[4]
daxTickers = list(table['Ticker'])
daxTickers[0] # adidas (ADS.DE)

'ADS.DE'

In [3]:
# Function to get history for a given ticker
def get_history(ticker, startDate = '2023-12-01', endDate = '2023-12-31'):
    df = yf.Ticker(ticker).history(start = startDate, end = endDate)
    df = df.drop(columns = ['Dividends', 'Volume', 'Stock Splits'])
    return df

In [4]:
# Function to fetch history for each ticker using threading
def get_histories(tickers, start_date='2023-12-01', end_date='2023-12-31'):
    dfs = []
    def worker(ticker):
        df = get_history(ticker, start_date, end_date)
        dfs.append((ticker,df))

    threads = [threading.Thread(target=worker, args=(ticker,)) for ticker in tickers]
    [thread.start() for thread in threads]
    [thread.join() for thread in threads]
    
    return dict(dfs)

In [5]:
data = get_histories(daxTickers)

In [6]:
def generateAlphaSignals(index, dfs = data, selection = 5):
    alphaSig = {}
    for k in dfs:
        #defining our formulaic alphas
        ################ 
        alphaSig[k] = (
            (dfs[k].iloc[index]["Close"] - dfs[k].iloc[index]["Open"]) /
            ((dfs[k].iloc[index]["High"] - dfs[k].iloc[index]["Low"]) + 0.001)
        )
        ################
    alphaSig = sorted(alphaSig.items(), key=lambda item: item[1])
    longSig = alphaSig[len(alphaSig)-selection:]
    shortSig = alphaSig[:selection]
    return longSig,shortSig

        
generateAlphaSignals(0)

([('DHL.DE', 0.8600417992394287),
  ('RHM.DE', 0.9300123159650997),
  ('MTX.DE', 0.9553439350717201),
  ('EOAN.DE', 0.9859154324187477),
  ('1COV.DE', 0.9995869472299371)],
 [('SRT3.DE', -0.8460693334274747),
  ('BNR.DE', -0.7091045674687763),
  ('BAYN.DE', -0.542021754246201),
  ('HNR1.DE', -0.4703142193079986),
  ('P911.DE', -0.43805598268355084)])

In [33]:

def backTest(capital = 10_000, dfs = data):
    min_days = np.inf
    for k in data:
        if len(data[k]) < min_days:
            min_days = len(data[k])
    
    for i in range(min_days-1):
        signals = generateAlphaSignals(i)
        for long in signals[0]:
            capital = capital * (1 +  ((data[long[0]].iloc[i+1]['Close'] - data[long[0]].iloc[i]['Close']) / data[long[0]].iloc[i]['Close']))
        for short in signals[1]:
            capital = capital * (1 +  ((data[short[0]].iloc[i]['Close'] - data[short[0]].iloc[i+1]['Close']) / data[short[0]].iloc[i]['Close']))
    return capital
backTest()

9506.75016344943